# Save sample data for testing

In [3]:
import boto3
import os
from tqdm import tqdm

s3 = boto3.client("s3")

# List all objects helper internal function
def list_all_objects(bucket, prefix):
    # Create a paginator for list_objects_v2
    paginator = s3.get_paginator('list_objects_v2')

    # Use the paginator to iterate through all pages
    all_objects = []
    for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
        if 'Contents' in page:
            all_objects.extend(page['Contents'])

    return all_objects

bucket = 'udacity-deeplearning-resubmit'
prefix = 'data'
sample_prefix = 'sample'

all_files = list_all_objects(bucket, prefix)

sample_files = [file_meta for file_meta in all_files if '001.Affenpinscher' in file_meta.get("Key")]

for file_meta in tqdm(sample_files):
    key = file_meta.get("Key")
    
    # Move the data from s3 to a different prefix
    dirname = os.path.dirname(key)
    if dirname:
        os.makedirs(dirname, exist_ok=True)

    s3.download_file(bucket, key, key)
    s3.copy_object(
        Bucket=bucket,
        CopySource=f"{bucket}/{key}",
        Key=f"{sample_prefix}/{key.replace('data/', '')}"
    )

100%|██████████| 80/80 [00:12<00:00,  6.39it/s]


## Testing Estimator Locally Prior to Deployment to ECR

In [1]:
from sagemaker.estimator import Estimator
from sagemaker import get_execution_role
import sagemaker
import subprocess

subprocess.run(['sh', 'docker-build.sh'])

role=get_execution_role()

bucket = 'udacity-deeplearning-project'
sample_prefix = 'sample'

hyperparameters = {
    'model-type': 'mobilenetv2',
    'num-classes': 133,
    'batch-size': 32,
    'lr': 0.005070970373087015
}

bucket = 'udacity-deeplearning-resubmit'
s3_output_location = f"s3://{bucket}/outputs"

estimator=Estimator(
    image_uri='udacity-sagemaker-hpo',
    role=role,
    instance_count=1,
    instance_type='local',
    output_path=s3_output_location,
    hyperparameters=hyperparameters
)

model_inputs = {
    "train": "file://./data/test", # Using test to test on a cheaper instance
    "test": "file://./data/valid"
}

estimator.fit(inputs=model_inputs, logs=True ,wait=True)

/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[12/20/24 13:23:37] INFO     Found credentials from IAM Role:                                   ]8;id=299399;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=865530;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded
sha256:0f017593f9978cdcf744759f51420a1becf3780d266357137a80008fc7a6d336
REPOSITORY              TAG       IMAGE ID       CREATED        SIZE
udacity-sagemaker-hpo   latest    0f017593f997   1 second ago   3.92GB


...

                    INFO     docker compose file:                                                      ]8;id=841072;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/local/image.py\image.py]8;;\:]8;id=362984;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/local/image.py#771\771]8;;\
                             networks:                                                                             
                               sagemaker-local:                                                                    
                                 name: sagemaker-local                                                             
                             services:                                                                             
                               algo-1-dw0to:                                                                       
                                 command: train                                                                    
                                 container_name: 1wr69fgudz-algo-1-dw0to                                           
                                 environment:                                                                      
                                 - '[Masked]'                                                                      
                                 - '[Masked]'                                                                      
                                 - '[Masked]'                                                                      
                                 image: udacity-sagemaker-hpo                                                      
                                 networks:                                                                         
                                   sagemaker-local:                                                                
                                     aliases:                                                                      
                                     - algo-1-dw0to                                                                
                                 stdin_open: true                                                                  
                                 tty: true                                                                         
                                 volumes:                                                                          
                                 - /tmp/tmpeut2_8dm/algo-1-dw0to/output/data:/opt/ml/output/data                   
                                 - /tmp/tmpeut2_8dm/algo-1-dw0to/input:/opt/ml/input                               
                                 - /tmp/tmpeut2_8dm/algo-1-dw0to/output:/opt/ml/output                             
                                 - /tmp/tmpeut2_8dm/model:/opt/ml/model                                            
                                 - /opt/ml/metadata:/opt/ml/metadata                                               
                                 -                                                                                 
                             /home/ec2-user/SageMaker/CD0387-deep-learning-topics-within-computer-visi             
                             on-nlp-project-starter/data/test:/opt/ml/input/data/train                             
                                 -                                                                                 
                             /home/ec2-user/SageMaker/CD0387-deep-learning-topics-within-computer-visi             
                             on-nlp-project-starter/data/valid:/opt/ml/input/data/test                             
                             version: '2.3'                                                                        
                        

                    INFO     docker command: docker-compose -f /tmp/tmpeut2_8dm/docker-compose.yaml up ]8;id=852325;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/local/image.py\image.py]8;;\:]8;id=425522;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/local/image.py#795\795]8;;\
                             --build --abort-on-container-exit                                                     

 Container 1wr69fgudz-algo-1-dw0to  Creating
 Container 1wr69fgudz-algo-1-dw0to  Created
Attaching to 1wr69fgudz-algo-1-dw0to
1wr69fgudz-algo-1-dw0to  | sed: can't read changehostname.c: No such file or directory
1wr69fgudz-algo-1-dw0to  | gcc: error: changehostname.c: No such file or directory
1wr69fgudz-algo-1-dw0to  | gcc: fatal error: no input files
1wr69fgudz-algo-1-dw0to  | compilation terminated.
1wr69fgudz-algo-1-dw0to  | gcc: error: changehostname.o: No such file or directory
1wr69fgudz-algo-1-dw0to  | ERROR: ld.so: object '/libchangehostname.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
1wr69fgudz-algo-1-dw0to  | 2024-12-20 13:25:48,090 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
1wr69fgudz-algo-1-dw0to  | 2024-12-20 13:25:48,093 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
1wr69fgudz-algo-1-dw0to  | 2024-12-20 13:25:48,094 sagemaker-training-toolkit INFO

[12/20/24 13:26:08] WARNING  Failed to delete: /tmp/tmpeut2_8dm/algo-1-dw0to Please remove it         ]8;id=691275;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/local/image.py\image.py]8;;\:]8;id=773926;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/local/image.py#1087\1087]8;;\
                             manually.                                                                             

                    INFO     ===== Job Complete =====                                                  ]8;id=296597;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/local/image.py\image.py]8;;\:]8;id=5470;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/local/image.py#325\325]8;;\

## Testing Deployed Estimator

In [2]:
!sh push-container.sh

push-container.sh: line 1: fg: no job control
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.1s (1/1) FINISHED                                 docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 2B                                         0.0s
ERROR: failed to solve: failed to read dockerfile: open Dockerfile: no such file or directory
The push refers to repository [477684614357.dkr.ecr.us-east-1.amazonaws.com/udacity-sagemaker-hpo]

bf18a086: Preparing 
fc893e27: Preparing 
1e9d3d7f: Preparing 
95045e04: Preparing 
e9708ca1: Preparing 
8f6060c6: Preparing 
a3c12226: Preparing 
62daa95e: Preparing 
8fe1cb59: Preparing 
061a5b0d:

In [3]:
model_inputs = {
    "train": sagemaker.inputs.TrainingInput(
        s3_data=f"s3://{bucket}/{sample_prefix}/test/", # To speed up the proces during this test and save on resources
        content_type="application/x-image"
    ),
    "test": sagemaker.inputs.TrainingInput(
        s3_data=f"s3://{bucket}/{sample_prefix}/valid/",
        content_type="application/x-image"
    )
}

In [4]:
hyperparameters = {
    'model-type': 'mobilenetv2',
    'num-classes': 133,
    'batch-size': 32,
    'lr': 0.005070970373087015,
    'epochs': 1
}

In [5]:
estimator=Estimator(
    image_uri='477684614357.dkr.ecr.us-east-1.amazonaws.com/udacity-sagemaker-hpo',
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    hyperparameters=hyperparameters
)

estimator.fit(inputs=model_inputs, logs=True)

[12/20/24 13:34:03] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=507790;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=602263;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#90\90]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=992976;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=395541;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\
                             udacity-sagemaker-hpo-2024-12-20-13-34-03-410                                         

2024-12-20 13:34:05 Starting - Starting the training job...
2024-12-20 13:34:18 Starting - Preparing the instances for training...
2024-12-20 13:35:00 Downloading - Downloading the training image......
2024-12-20 13:36:02 Training - Training image download completed. Training in progress.
2024-12-20 13:36:02 Uploading - Uploading generated training modelbash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
sed: can't read changehostname.c: No such file or directory
gcc: error: changehostname.c: No such file or directory
gcc: fatal error: no input files
compilation terminated.
gcc: error: changehostname.o: No such file or directory
ERROR: ld.so: object '/libchangehostname.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
2024-12-20 13:35:54,398 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-12-20 13:35:54,401 sagemaker-training-toolkit INFO     No 

In [6]:
import boto3

sagemaker_client = boto3.client("sagemaker")

# Get the latest training job name
training_job_name = estimator.latest_training_job.name
print(f"Training Job Name: {training_job_name}")

# Get the model artifact location from the training job details
response = sagemaker_client.describe_training_job(TrainingJobName=training_job_name)
model_artifact = response["ModelArtifacts"]["S3ModelArtifacts"]
print(f"Model Artifact Location: {model_artifact}")


Training Job Name: udacity-sagemaker-hpo-2024-12-20-13-34-03-410
Model Artifact Location: s3://sagemaker-us-east-1-477684614357/udacity-sagemaker-hpo-2024-12-20-13-34-03-410/output/model.tar.gz


# Testing Batch Inference

In [28]:
from sagemaker.pytorch import PyTorchModel

pytorch_model = PyTorchModel(
    model_data=model_artifact,  # Use the artifact location from the training job
    role=estimator.role,       # Use the role from your estimator
    entry_point="image/inference.py",
    framework_version="1.8.1",
    py_version="py3",
    image_uri="763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.8.1-cpu-py36-ubuntu18.04",  # PyTorch-Inference image
)


In [29]:
transformer = pytorch_model.transformer(
    instance_count=1,
    instance_type="ml.m5.large",
    output_path="s3://udacity-deeplearning-project/inference/",  # S3 location for output
)


INFO:sagemaker:Repacking model artifact (s3://sagemaker-us-east-1-598308907998/udacity-sagemaker-hpo-2024-11-24-20-20-00-150/output/model.tar.gz), script artifact (None), and dependencies ([]) into single tar.gz file located at s3://sagemaker-us-east-1-598308907998/pytorch-inference-2024-11-24-20-26-25-124/model.tar.gz. This may take some time depending on model size...
INFO:sagemaker:Creating model with name: pytorch-inference-2024-11-24-20-26-29-479


In [30]:
transformer.transform(
    data="s3://udacity-deeplearning-project/sample/data/batch/",
    content_type="application/x-image",
    wait=True
)

print("Batch transform completed. Check the output in the specified S3 bucket.")

INFO:sagemaker:Creating transform job with name: pytorch-inference-2024-11-24-20-26-32-239


..............................2024-11-24 20:31:35,030 [INFO ] main org.pytorch.serve.servingsdk.impl.PluginsManager - Initializing plugins manager...
2024-11-24 20:31:35,237 [INFO ] main org.pytorch.serve.ModelServer - 
Torchserve version: 0.4.0
TS Home: /opt/conda/lib/python3.6/site-packages
Current directory: /
Temp directory: /home/model-server/tmp
Number of GPUs: 0
Number of CPUs: 2
Max heap size: 952 M
Python executable: /opt/conda/bin/python3.6
Config file: /etc/sagemaker-ts.properties
Inference address: http://0.0.0.0:8080
Management address: http://0.0.0.0:8080
Metrics address: http://127.0.0.1:8082
Model Store: /.sagemaker/ts/models
Initial Models: model.mar
Log dir: /logs
Metrics dir: /logs
Netty threads: 0
Netty client threads: 0
Default workers per model: 2
Blacklist Regex: N/A
Maximum Response Size: 6553500
Maximum Request Size: 6553500
Prefer direct buffer: false
Allowed Urls: [file://.*|http(s)?://.*]
Custom python dependency for model allowed: false
Metrics report forma

# END OF NOTEBOOK